In [105]:
%matplotlib inline
import re
import os
import numpy as np
import pandas as pd
import seaborn as sns
from requests import get
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# Import Data

In [106]:
dfList = []
for r, d, f in os.walk('../data/dunnhumby - The Complete Journey CSV/'):
    for file in f:
        if '.csv' in file:
            print(file)
            dfList.append(pd.read_csv(os.path.join(r, file)))

In [107]:
campaign_desc_df = dfList[0]
campaign_table_df = dfList[1]
causal_data_df = dfList[2]
coupon_df = dfList[3]
coupon_redempt_df = dfList[4]
hh_demographic_df = dfList[5]
product_df = dfList[6]
transaction_data_df = dfList[7]

IndexError: list index out of range

# Observing Data

In [108]:
campaign_desc_df.head(2)

NameError: name 'campaign_desc_df' is not defined

In [109]:
campaign_table_df.head(2)

NameError: name 'campaign_table_df' is not defined

In [110]:
causal_data_df.head(2)

NameError: name 'causal_data_df' is not defined

In [111]:
coupon_df.head(2)

NameError: name 'coupon_df' is not defined

In [99]:
coupon_redempt_df.head(2)

,household_key,DAY,COUPON_UPC,CAMPAIGN
0,1,421,10000085364,8
1,1,421,51700010076,8


In [100]:
hh_demographic_df.head(2)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


This seems to be interesting data about households

## Product Data

In [101]:
product_df.head(10)

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
5,26426,69,GROCERY,Private,SPICES & EXTRACTS,SPICES & SEASONINGS,2.5 OZ
6,26540,69,GROCERY,Private,COOKIES/CONES,TRAY PACK/CHOC CHIP COOKIES,16 OZ
7,26601,69,DRUG GM,Private,VITAMINS,VITAMIN - MINERALS,300CT(1)
8,26636,69,PASTRY,Private,BREAKFAST SWEETS,SW GDS: SW ROLLS/DAN,
9,26691,16,GROCERY,Private,PNT BTR/JELLY/JAMS,HONEY,12 OZ


In [103]:
product_df[product_df['DEPARTMENT']=='GROCERY'].groupby('COMMODITY_DESC')['PRODUCT_ID'].count().sort_values(ascending=False)

COMMODITY_DESC
SOFT DRINKS               1704
BAG SNACKS                1523
HISPANIC                  1460
FRZN MEAT/MEAT DINNERS    1268
BAKED BREAD/BUNS/ROLLS    1169
                          ... 
BIRD SEED                   29
FROZEN CHICKEN              27
COUPON/MISC ITEMS           17
FRZN SEAFOOD                11
BOTTLE DEPOSITS             10
Name: PRODUCT_ID, Length: 94, dtype: int64

Let us look at what kind of 'Drug GM' products we have

In [104]:
product_df[product_df['DEPARTMENT']=='DRUG GM'].groupby('COMMODITY_DESC')['PRODUCT_ID'].count().sort_values(ascending=False)

COMMODITY_DESC
GREETING CARDS/WRAP/PARTY SPLY    2785
CANDY - PACKAGED                  2473
HAIR CARE PRODUCTS                1744
STATIONERY & SCHOOL SUPPLIES      1261
MAGAZINE                          1224
                                  ... 
COUPON/MISC ITEMS                    5
PROPANE                              3
MISCELLANEOUS HBC                    3
FRAGRANCES                           1
FD WRAPS/BAGS/TRSH BG                1
Name: PRODUCT_ID, Length: 91, dtype: int64

# Downloaded food nutrients data

In [112]:
dfList = []
for r, d, f in os.walk('../data/'):
    for file in f:
        if '.csv' in file:
            print(file)
            dfList.append(pd.read_csv(os.path.join(r, file)))
            
branded_food_df = dfList[0]
nutrient_df = dfList[1]
food_df = dfList[2]
food_nutrients_df = dfList[3]
food_category_df = dfList[4]

branded_food.csv
nutrient.csv
food.csv
food_nutrient.csv
food_category.csv


In [113]:
#drop unnecessary columns and rename to be more understandable
food_nutrients_small_df = food_nutrients_df.drop(["data_points","min","max","median","footnote","min_year_acquired","derivation_id"],axis=1)
nutrient_small_df = nutrient_df.drop(["nutrient_nbr","rank"],axis=1)
food_category_df.drop(["code"],axis=1,inplace=True)
food_category_df.rename(columns={'id':'food_category_id','description':'category'},inplace= True)
food_df.drop(["publication_date"],axis=1,inplace=True)

In [114]:
# join the df to see which id corresponds to which nutrients
joined_df = food_nutrients_small_df.join(nutrient_small_df.set_index('id'),on='nutrient_id',how='left')
joined_df.head()

,id,fdc_id,nutrient_id,amount,name,unit_name
0,4178832,346049,1079,0.00,"Fiber, total dietary",G
1,4178833,346049,1087,0.00,"Calcium, Ca",MG
2,4178834,346049,1089,2.57,"Iron, Fe",MG
3,4178835,346049,1104,0.00,"Vitamin A, IU",IU
4,4178836,346049,1162,0.00,"Vitamin C, total ascorbic acid",MG


In [123]:
#index the resulting table by multiindex: product id -> name of nutrients
structured_df = joined_df.set_index(pd.MultiIndex.from_frame(joined_df[['fdc_id','nutrient_id']]))
#drop unnecessary columns 
structured_small_df = structured_df.drop(["id","fdc_id","nutrient_id","name"],axis=1)

In [124]:
structured_small_df.head()

amount unit_name
fdc_id nutrient_id                  
346049 1079           0.00         G
       1087           0.00        MG
       1089           2.57        MG
       1104           0.00        IU
       1162           0.00        MG

In [125]:
food_df.head()

,fdc_id,data_type,description,food_category_id
0,346049,branded_food,"LIBBYS Corned Beef With Onion, 12 OZ",NaN
1,346050,branded_food,"LIBBYS Corned Beef With Chili, 12 OZ",NaN
2,346464,branded_food,WOLF Chili Without Beans,NaN
3,346466,branded_food,"WOLF Turkey Chili No Beans, 15 OZ",NaN
4,346468,branded_food,"WOLF BRAND Chili With Beans, 24 oz., 24 OZ",NaN


In [126]:
food_with_category_df = food_df.join(food_category_df.set_index("food_category_id"),on="food_category_id",how="left")
food_with_category_df.drop(["food_category_id"],axis=1,inplace=True)

In [127]:
food_with_category_df.head()

,fdc_id,data_type,description,category
0,346049,branded_food,"LIBBYS Corned Beef With Onion, 12 OZ",NaN
1,346050,branded_food,"LIBBYS Corned Beef With Chili, 12 OZ",NaN
2,346464,branded_food,WOLF Chili Without Beans,NaN
3,346466,branded_food,"WOLF Turkey Chili No Beans, 15 OZ",NaN
4,346468,branded_food,"WOLF BRAND Chili With Beans, 24 oz., 24 OZ",NaN


In [128]:
structured_name_df = structured_small_df.join(food_with_category_df.set_index('fdc_id'),on='fdc_id',how='left')

In [129]:
structured_name_df.head()

amount unit_name     data_type  \
fdc_id nutrient_id                                   
346049 1079           0.00         G  branded_food   
       1087           0.00        MG  branded_food   
       1089           2.57        MG  branded_food   
       1104           0.00        IU  branded_food   
       1162           0.00        MG  branded_food   

                                             description category  
fdc_id nutrient_id                                                 
346049 1079         LIBBYS Corned Beef With Onion, 12 OZ      NaN  
       1087         LIBBYS Corned Beef With Onion, 12 OZ      NaN  
       1089         LIBBYS Corned Beef With Onion, 12 OZ      NaN  
       1104         LIBBYS Corned Beef With Onion, 12 OZ      NaN  
       1162         LIBBYS Corned Beef With Onion, 12 OZ      NaN

In [146]:
structured_rows_df = structured_name_df.unstack(level=1,fill_value=0)

In [147]:
structured_rows_df.columns

MultiIndex([(  'amount', 1002),
            (  'amount', 1003),
            (  'amount', 1004),
            (  'amount', 1005),
            (  'amount', 1007),
            (  'amount', 1008),
            (  'amount', 1009),
            (  'amount', 1010),
            (  'amount', 1011),
            (  'amount', 1012),
            ...
            ('category', 2020),
            ('category', 2021),
            ('category', 2022),
            ('category', 2023),
            ('category', 2024),
            ('category', 2025),
            ('category', 2026),
            ('category', 2028),
            ('category', 2029),
            ('category', 2032)],
           names=[None, 'nutrient_id'], length=1135)

In [148]:
structured_rows_df.head(1)

amount                                                      ...  \
nutrient_id   1002  1003   1004   1005 1007   1008 1009 1010 1011 1012  ...   
fdc_id                                                                  ...   
167512         0.0  5.88  13.24  41.18  3.5  307.0  0.0  0.0  0.0  0.0  ...   

            category                                               
nutrient_id     2020 2021 2022 2023 2024 2025 2026 2028 2029 2032  
fdc_id                                                             
167512             0    0    0    0    0    0    0    0    0    0  

[1 rows x 1135 columns]

In [145]:
structured_name_df.index.get_level_values(1)

Int64Index([1079, 1087, 1089, 1104, 1162, 1003, 1004, 1005, 1092, 1093,
            ...
            1299, 1304, 1305, 1306, 1311, 1313, 1314, 1321, 1325, 1406],
           dtype='int64', name='nutrient_id', length=4800957)

In [149]:
test_df = joined_df.head(30)

,id,fdc_id,nutrient_id,amount,name,unit_name
0,4178832,346049,1079,0.00,"Fiber, total dietary",G
1,4178833,346049,1087,0.00,"Calcium, Ca",MG
2,4178834,346049,1089,2.57,"Iron, Fe",MG
3,4178835,346049,1104,0.00,"Vitamin A, IU",IU
4,4178836,346049,1162,0.00,"Vitamin C, total ascorbic acid",MG


In [ ]:
joined_df.pivot(columns='nutrient_id')